# The War Of The Five Kings

this notebook is based in the analysis made in https://github.com/chrisalbon/war_of_the_five_kings_dataset, and is based on the dataset of the battles in the War of the Five Kings from George R.R. Martin's A Song Of Ice And Fire series.



### 1. import libraries

In [ ]:
import "os"
import "fmt"
import "encoding/csv"
import "gonum.org/v1/plot"
import "gonum.org/v1/plot/plotter"
import "gonum.org/v1/plot/plotutil"
import "gonum.org/v1/plot/vg"
import "io/ioutil"
import "strings"




### 2. open the csv file ../../data/5kings_battles_v1.csv

In [ ]:
file, err := os.Open("../data/5kings_battles_v1.csv")
if err != nil {
    fmt.Println(err)
}

### 3. load csv file

In [ ]:
gotcsv := csv.NewReader(file)

In [ ]:
records, err := gotcsv.ReadAll()
if err != nil {
    fmt.Println(err)
}

In [ ]:
for _, record := range records {
    fmt.Println(record[0],"\t",record[1],":",record[3],record[4],record[5])
}

#### ... don't forget to close the file

In [ ]:
file.Close()

In [ ]:
fmt.Println(records[0])

### 4. prepare the data for the analisys, transpose the matrix to handle it 

In [ ]:
func  transpose(matrix [][]string) [][]string {
    r := make([][]string, len(matrix[0])) 
    for index :=range r{
        r[index] = make([]string, len(matrix))
    }
    for fv, f := range matrix {
        for cv, c :=range f {
            r[cv][fv] = c
        }
    }
    return r
}

In [ ]:
t:=transpose(records)

In [ ]:
fmt.Println(len(records))

In [ ]:
fmt.Println(t[0])

# Exploratory Data Analysis

## Which year had the most battles?

### Count the number of observations for each value

In [ ]:
fmt.Println(t[1][1:])

In [ ]:
func countObservations(vector []string )map[string]int{
    freq := make(map[string]int)
    for _, item := range vector{
        freq[item]++
    }
    return freq
}

In [ ]:
yearCount := countObservations(t[1][1:])

In [ ]:
fmt.Println(yearCount)

## Which region had the most battles?

### Count the number of observations for each value, then make a bar plot

In [ ]:
regionCount := countObservations(t[21][1:])

In [ ]:
fmt.Println(regionCount)

In [ ]:
p, err := plot.New()
if err != nil {
    fmt.Println(err)
}

In [ ]:
p.Title.Text = "observations for each value"
p.Y.Label.Text = "count"

In [ ]:
func splitKeysAndValues(mapData map[string]int)([]string,plotter.Values){
    values := make(plotter.Values, len(mapData))
    keys := make([]string, len(mapData))
    idx := 0
    for  key, value := range mapData {
        values[idx] = float64(value)
        keys[idx] = strings.Replace(key," ","\n",-1)
           idx++
    }
    return keys,values
}

In [ ]:
keys,values := splitKeysAndValues(regionCount)

In [ ]:
fmt.Println(values)

In [ ]:
fmt.Println(keys)

In [ ]:
w := vg.Points(20)

In [ ]:
barsA, err := plotter.NewBarChart(values, w)
if err != nil {
	fmt.Println(err)
}

In [ ]:
p.Add(barsA)

In [ ]:
p.NominalX(keys...)

In [ ]:
if err := p.Save(5*vg.Inch, 3*vg.Inch, "barchart3.png"); err != nil {
	fmt.Println(err)
}

In [ ]:
func getGraph(graphName string)[]byte{
    infile, err := os.Open(graphName)
	bytes, err := ioutil.ReadAll(infile)
	infile.Close()
    return bytes
}

In [ ]:
graph:=getGraph("barchart3.png")

In [ ]:
display.PNG(graph)

## Is there any relationship between troop size and battle outcome?

### Create a ggplot scatter plot of attacker_size against defender_size (if not NaN), 
### with the color of each dot being determined by the outcome of the battle

In [ ]:
attackerSize := t[15][1:]

In [ ]:
deffenderSize := t[16][1:]

In [ ]:
fmt.Println(attackerSize)

In [ ]:
fmt.Println(deffenderSize)

In [ ]:
attackerOutcome :=t[11][1:]

In [ ]:
fmt.Println(attackerOutcome)

In [ ]:
var attackerWins [2][]string
var attackerLose [2][]string
attackerWins[0] = make([]string, len(attackerOutcome))
attackerWins[1] = make([]string, len(attackerOutcome))
attackerLose[0] = make([]string, len(attackerOutcome))
attackerLose[1] = make([]string, len(attackerOutcome))
atwins :=0
atLose := 0
for i,val := range attackerOutcome {
    if val=="win"{
        attackerWins[0][atwins]=attackerSize[i]
        attackerWins[1][atwins]=deffenderSize[i]
        atwins++
    }else{
        attackerLose[0][atLose]=attackerSize[i]
        attackerLose[1][atLose]=deffenderSize[i]
        atLose++
    }
}

In [ ]:
troopvsoutcome, err := plot.New()

In [ ]:
troopvsoutcome.Title.Text = "relationship between troop size and battle outcome"
troopvsoutcome.Y.Label.Text = "attacker size"
troopvsoutcome.X.Label.Text = "defffender size"

In [ ]:
troopvsoutcome.Add(plotter.NewGrid())

In [ ]:
import "strconv"
import "image/color"

In [ ]:
func attackerVsDeffenderSize(ndeffender []string, natacker []string) plotter.XYs {
    pts := make(plotter.XYs, len(ndeffender))
	for i := range pts {
        if ndeffender[i] == ""{
            ndeffender[i]="0"
        }
        if natacker[i] == ""{
            natacker[i]="0"
        }
        pts[i].X = strconv.ParseFloat(ndeffender[i], 64)
        pts[i].Y = strconv.ParseFloat(natacker[i], 64)
	}
	return pts
}

In [ ]:
s, err := plotter.NewScatter(attackerVsDeffenderSize(attackerWins[1],attackerWins[0]))
if err != nil{
    fmt.Println(err)
}
s.GlyphStyle.Color = color.RGBA{R: 255, B: 128, A: 255}

In [ ]:
s1, err := plotter.NewScatter(attackerVsDeffenderSize(attackerLose[1],attackerLose[0]))
if err != nil {
    fmt.Println(err)
}
s1.GlyphStyle.Color = color.RGBA{R: 0, B: 128, A: 255}

In [ ]:
troopvsoutcome.Add(s,s1)

In [ ]:
if err := troopvsoutcome.Save(4*vg.Inch, 4*vg.Inch, "points.png"); err != nil {
	fmt.Println(err)
}

In [ ]:
graph1:=getGraph("points.png")

In [ ]:
display.PNG(graph1)